## Overview

Kamodo provides a *functional* interface for space weather analysis, visualization, and knowledge discovery, allowing many problems in scientific data analysis to be posed in terms of function composition and evaluation. We'll walk through its general features here.

##  Kamodo objects

Users primarily interact with models and data through Kamodo objects. 

In [ ]:
from kamodo import Kamodo

### Function registration
Kamodo objects are essentially python dictionaries storing variable symbols as keys and their interpolating functions as values. New functions may be registered either at the initialization of the Kamodo object or later using dictionary bracket syntax.

In [ ]:
kamodo = Kamodo(f = 'x**2')
kamodo

In [ ]:
kamodo.f(3)

In [ ]:
kamodo = Kamodo('$x = t^2$')
kamodo['g'] = 'y-1'
kamodo

In [ ]:
kamodo.g(3)

### Function composition
Kamodo automatically composes functions through specifying on the right-hand-side.

In [ ]:
kamodo['f'] = 'g(x)'
kamodo

Here we have defined two functions $x(t)$, $g(y)$, and the composition $g∘f$. Kamodo was able to determine that $f$ is implicitly a function of $t$ even though we did not say so in $f$'s declaration.

#### Function evaluation
Kamodo uses sympy's ```lambdify``` function to turn the above equations into highly optimized functions for numerical evaluation. We may evaluate $f(t)$ for $t=3$ using "dot" notation:

In [ ]:
kamodo.f(3)

In [ ]:
help(kamodo.g)

where the return type is a numpy array. We could also have passed in a numpy array and the result shares the same shape:

In [ ]:
import numpy as np
t = np.linspace(-5, 5, 100000)
result = kamodo.f(t)

In [ ]:
assert(t.shape == result.shape)

### Unit conversion
Kamodo automatically handles unit conversions. Simply declare units on the left-hand-side of expressions using bracket notation.

In [ ]:
kamodo = Kamodo('mass[kg] = x', 'vol[m^3] = y')

In [ ]:
kamodo

Unless specified, Kamodo will assign the units for newly defined variables:

In [ ]:
kamodo['rho'] = 'mass/vol'
kamodo

We may override the default behavior by simply naming the our chosen units in the left hand side.

In [ ]:
kamodo['rho(x,y)[g/cm^3]'] = 'mass/vol'
kamodo

!!! note
    Kamodo will raise an error if the left and right-hand-side units are incompatible.

Even though generated functions are unitless, the units are clearly displayed on the lhs. We think this is a good trade-off between performance and legibility.

In [ ]:
kamodo.detail()

In [ ]:
kamodo.rho(3,4)

We can verify that kamodo produces the correct output upon evaluation.

In [ ]:
assert(kamodo.rho(3,8) == (3*1000.)/(8*100**3))

### Variable naming conventions
Kamodo allows for a wide array of variable names to suite your problem space, including greek, subscripts, superscripts.

In [ ]:
kamodo = Kamodo(
    'rho = ALPHA+BETA+GAMMA',
    'rvec = t',
    'fprime = x',
    'xvec_i = xvec_iminus1 + 1',
    'F__gravity = G*M*m/R**2',
)
kamodo

In [ ]:
def rho(x):
    pass
Kamodo(rho = rho)

For more details on variable names, see the [Syntax](../Syntax/) section.

## Kamodofication

Many functions can not be written as simple mathematical expressions - they could represent simulation output or observational data. For this reason, we provide a ```@kamodofy``` decorator, which turns any callable function into a kamodo-compatible variable and adds metadata that enables unit conversion.

In [ ]:
from kamodo import kamodofy, Kamodo
import numpy as np

@kamodofy(units = 'kg/m**3', citation = 'Bob et. al, 2018')
def rho(x = np.array([3,4,5]), y = np.array([1,2,3])):
    """A function that computes density"""
    return x+y

kamodo = Kamodo(rho = rho)
kamodo['den[g/cm^3]'] = 'rho'
kamodo

In [ ]:
kamodo.den(3,4)

In [ ]:
kamodo.rho.meta # PyHC standard

In [ ]:
kamodo.rho()

In [ ]:
kamodo.rho.data # PyHC standard

Original function doc strings and signatures passed through

In [ ]:
help(kamodo.rho)

In [ ]:
kamodo.detail()

# Visualization

Kamodo graphs are generated directly from function signatures by examining the structure of both output and input arguments. 

In [ ]:
from plotting import plot_types
plot_types

Kamodo uses [plotly](https://plot.ly/python/) for visualization, enabling a rich array of interactive graphs and easy web deployment.

In [ ]:
import plotly.io as pio
from plotly.offline import iplot,plot, init_notebook_mode
init_notebook_mode(connected=True)

In [ ]:
@kamodofy(units = 'kg/m^3')
def rho(x = np.linspace(0,1, 20), y = np.linspace(-1, 1, 40)):
    """A function that computes density"""
    x_, y_ = np.meshgrid(x,y)
    return x_*y_

kamodo = Kamodo(rho = rho)
kamodo

We will generate an image of this function using plotly

In [ ]:
fig = kamodo.plot('rho')
# pio.write_image(fig, 'images/Kamodo_fig1.svg')

![fig1](images/Kamodo_fig1.svg)

See the [Visualization](../Visualization/) section for detailed examples.

## Latex I/O

Even though math is the language of physics, most scientific analysis software requires you to learn new programing languages. Kamodo allows users to write their mathematical expressions in LaTeX, a typesetting language most scientists already know:

In [ ]:
kamodo = Kamodo('$rho[kg/m^3] = x^3$', '$v[cm/s] = y^2$')
kamodo['p[Pa]'] = '$\\rho v^2$'
kamodo

The resulting equation set may also be exported as a LaTeX string for use in publications:

In [ ]:
print(kamodo.to_latex() + '\n.')

# Simulation api

Kamodo offers a simple api for functions composed of each other.

Define variables as usual (order matters).

In [ ]:
kamodo = Kamodo()
kamodo['y_iplus1'] = 'x_i + 1'
kamodo['x_iplus1'] = 'y_i - 2'
kamodo

Now add the ```update``` attribute to map functions onto arguments.

In [ ]:
kamodo.x_iplus1.update = 'x_i'
kamodo.y_iplus1.update = 'y_i'

Create a simulation with initial conditions

In [ ]:
simulation = kamodo.simulate(x_i = 0, steps = 5)
simulation #an iterator of arg, val dictionaries

Run the simulation by iterating through the generator.

In [ ]:
import pandas as pd
pd.DataFrame(simulation) # pandas conveniently iterates the results for display